In [1]:
import urllib
import os
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import ipywidgets as widgets



In [2]:
FILES_URL = 'https://datasette.planning.data.gov.uk/'

def download_dataset(dataset, output_dir_path, overwrite=False):
    dataset_file_name = f'{dataset}.db'
    
    if not os.path.exists(output_dir_path):
        os.makedirs(output_dir_path)
    
    output_file_path = os.path.join(output_dir_path, dataset_file_name)

    if overwrite is False and os.path.exists(output_file_path):
        return
    
    final_url = os.path.join(FILES_URL, dataset_file_name)
    print(f'downloading data from {final_url}')
    print(f'to: {output_file_path}')
    urllib.request.urlretrieve(final_url, os.path.join(output_dir_path, dataset_file_name))
    print('download complete')

In [3]:
def query_sqlite(db_path, query_string):

    with sqlite3.connect(db_path) as con:
            
        cursor = con.execute(query_string)
        cols = [column[0] for column in cursor.description]
        results_df = pd.DataFrame.from_records(data=cursor.fetchall(), columns=cols)

    return results_df

In [4]:

def datasette_query(db, sql_string):
    params = urllib.parse.urlencode({
        "sql": sql_string,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/{db}.csv?{params}"
    df = pd.read_csv(url)
    return df

def get_issue_lookup():
    
    q = """
    select issue_type, severity, responsibility
    from issue_type
"""
    return datasette_query("digital-land", q)

In [5]:
data_dir = "../../data/db_downloads/"
os.makedirs(data_dir, exist_ok=True)

## Get data

In [6]:
download_dataset("performance", data_dir, overwrite=True)
# download_dataset("article-4-direction-area", data_dir, overwrite=True)
# download_dataset("tree-preservation-zone", data_dir, overwrite=True)

downloading data from https://datasette.planning.data.gov.uk/performance.db
to: ../../data/db_downloads/performance.db
download complete


In [7]:
lookup_issue_qual = pd.read_csv("data/issue_type_quality.csv")

In [8]:
dataset_subset_dict = dict({
        "ODP" : ["conservation-area", "conservation-area-document", "article-4-direction-area", "article-4-direction", "listed-building-outline", "tree", "tree-preservation-zone", "tree-preservation-order"],
        "BFL" : ["brownfield-land"],
        "Developers" : ["developer-agreement", "developer-agreement-contribution", "developer-agreement-transaction"]
    })

In [9]:
# BASE TABLE

# get table of active endpoints and resources, with issue summaries per resource joined on
q = f"""
    SELECT 
        rhe.organisation, rhe.name as organisation_name, 
        rhe.collection, rhe.pipeline, rhe.endpoint, rhe.resource, rhe.latest_status, rhe.endpoint_entry_date, rhe.resource_start_date, 
        CAST(JULIANDAY('now') - JULIANDAY(rhe.resource_start_date) AS int) as resource_age_days,
        its.issue_type, its.count_issues, its.severity, its.responsibility
    FROM reporting_historic_endpoints rhe
    LEFT JOIN endpoint_dataset_issue_type_summary its on rhe.resource = its.resource
    WHERE 1=1
        AND rhe.endpoint_end_date = ""
        AND rhe.resource_end_date = ""
        AND rhe.latest_status = 200
"""

ep_res_issues = query_sqlite(os.path.join(data_dir, "performance.db"), q)

print(len(ep_res_issues))
ep_res_issues.head()

2891


,organisation,organisation_name,collection,pipeline,endpoint,resource,latest_status,endpoint_entry_date,resource_start_date,resource_age_days,issue_type,count_issues,severity,responsibility
0,development-corporation:Q20648596,Old Oak and Park Royal Development Corporation,brownfield-land,brownfield-land,a16e45dbefe2d67a6d27c086768b6c3610d4e057bb1962...,f1e218c96f99e378fdbaed9a426c6b44d0e7d3b5fec63e...,200,2019-12-01,2019-12-01,1804,patch,1.0,info,internal
1,development-corporation:Q20648596,Old Oak and Park Royal Development Corporation,brownfield-land,brownfield-land,a16e45dbefe2d67a6d27c086768b6c3610d4e057bb1962...,f1e218c96f99e378fdbaed9a426c6b44d0e7d3b5fec63e...,200,2019-12-01,2019-12-01,1804,default-field,41.0,info,internal
2,government-organisation:D1342,"Ministry of Housing, Communities & Local Gover...",area-of-outstanding-natural-beauty,area-of-outstanding-natural-beauty,34167c688ca765477c47e63592ff9149bb46710a0850f6...,d4f9a4c7d1bda9026c2079812c5f2fc9e6de231b111b7e...,200,2022-09-10,2024-10-03,36,None,NaN,None,None
3,government-organisation:D1342,"Ministry of Housing, Communities & Local Gover...",article-4-direction,article-4-direction-rule,e5598a75328d5890efb7e6d8bb6c1d181fbed2cda45af2...,44938e5a202309da506987cb84eb51fe789993f0870a9a...,200,2022-10-11,2022-10-25,745,None,NaN,None,None
4,government-organisation:D1342,"Ministry of Housing, Communities & Local Gover...",conservation-area,conservation-area,10e4c16fddeb43fe3a1f2d612ffc12418a6bac2ef28782...,c567bc3c40dcc06d7cc9048e6bb4b86ceae58274ccb963...,200,2024-10-17,2024-10-25,14,None,NaN,None,None


In [10]:
# Provision lookups

q = f"""
    SELECT 
        distinct organisation, pipeline, cohort
    FROM endpoint_dataset_resource_summary
"""

# get organisation, pipeline and cohort flag from performance table
lookup_provision = query_sqlite(os.path.join(data_dir, "performance.db"), q)

# lists for ODP datasets and orgs
lookup_odp_datasets = lookup_provision[lookup_provision["cohort"].str.contains("ODP")]["pipeline"].drop_duplicates().to_list()
lookup_odp_orgs = lookup_provision[lookup_provision["cohort"].str.contains("ODP")]["organisation"].drop_duplicates().to_list()

## Setup base tables

In [11]:
# FRESHNESS TABLE - flagging when provisions haven't been updated in last year

# create table of old resources and flag quality level as 5
ep_res_fresh_qual = ep_res_issues[ep_res_issues["resource_age_days"] > 365][["collection", "pipeline", "organisation", "organisation_name"]]

ep_res_fresh_qual["issue_type"] = "not_fresh"
ep_res_fresh_qual["quality_category"] = "1 - endpoint not updated in last year"
ep_res_fresh_qual["quality_level"] = 1

In [12]:
# ISSUES TABLE - flagging when provisions have data quality issues

# join on quality key and restrict fields
ep_res_issues_qual = ep_res_issues.merge(
    lookup_issue_qual[["issue_type", "quality_category", "quality_level"]],
    how = "left",
    on = "issue_type"
)[["collection", "pipeline", "organisation", "organisation_name", "issue_type", "quality_category", "quality_level"]]

print(len(ep_res_issues))
print(len(ep_res_issues_qual))

# ep_res_issues_qual.head()

2891
2891


In [13]:
# ALL QUALITY CATEGORIES TABLE - joining all records of quality categories (freshness & DQ issues) into one long table 
# concat tables for each type
ep_res_qual_all = pd.concat([ep_res_issues_qual, ep_res_fresh_qual])


In [14]:
# SCORING - using quality framework levels to assign a quality level to each provision

# summarise by provision, taking max quality level for each
qual_summary = ep_res_qual_all.groupby([
    "collection", "pipeline", "organisation", "organisation_name"
    ],
as_index=False,
dropna=False
).agg(
    quality_level = ("quality_level", "min")
)

qual_summary.replace(np.nan, 4, inplace=True)

level_map = {
    4: "4. excellent",
    3: "3. good for ODP",
    2: "2. improve",
    1: "1. update"}

qual_summary["quality_level_label"] = qual_summary["quality_level"].map(level_map)
print(len(qual_summary))

697


## LPA and dataset table summaries

### ODP LPA x Dataset quality table

In [15]:
# qual_summary

lookup_provision_odp = lookup_provision[
    lookup_provision["cohort"].str.contains("ODP")
    ][["organisation", "pipeline"]].drop_duplicates()

 # subset and pivot
odp_lpa_summary = qual_summary.merge(
    lookup_provision_odp,
    how = "inner",
    on = ["organisation", "pipeline"]
).pivot(
    columns = "pipeline",
    values = "quality_level_label",
    index = ["organisation", "organisation_name"]
).reset_index()

odp_lpa_summary.replace(np.nan, "no data", inplace=True)
# odp_lpa_summary

In [16]:
level_colours = {
    "4. excellent" : "background-color: #1a6837",
    "3. good for ODP" : "background-color: #87cb67",
    "2. improve" : "background-color: #fefebf",
    "1. update" : "background-color: #f78c51"
    }


def make_color_mask_odp_lpa(df):
    #DataFrame with same index and columns names as original filled empty strings
    df_color_map =  pd.DataFrame("", index=df.index, columns=df.columns)

    flag_slice = df.columns[2:]
    for s in flag_slice:
        df_color_map[s] = df[s].map(level_colours)

    return df_color_map

# odp_lpa_summary.style.apply(make_color_mask_odp_lpa, axis=None)

### Dataset x quality categories table

In [17]:
# count issues by the quality category key
qual_cat_count = ep_res_qual_all.groupby(
        ["pipeline", "organisation", "organisation_name", "quality_category"],
        as_index=False
    ).agg(
        n_issues = ("quality_level", "count")
    )

In [18]:
# create a base table with each quality category key for each provision
prov = ep_res_qual_all[["pipeline", "organisation", "organisation_name"]].drop_duplicates()
prov["key"] = 1

qual_cat = ep_res_qual_all[ep_res_qual_all["quality_category"].notnull()][["quality_category"]].drop_duplicates()
qual_cat["key"] = 1

qual_cat_summary = prov.merge(
    qual_cat,
    how = "left",
    on = "key"
)
print(len(qual_cat_summary))

# left join on the counts to the base table
qual_cat_summary = qual_cat_summary.merge(
    qual_cat_count,
    how = "left",
    on = ['pipeline', 'organisation', 'organisation_name', 'quality_category']
)

# create boolean flag for each category
qual_cat_summary["issue_flag"] = np.where(qual_cat_summary["n_issues"] > 0, True, False)
print(len(qual_cat_summary))
# qual_cat_summary.head()

3485
3485


In [19]:

level_colours = {
        "4. excellent" : "background-color: #1a6837",
        "3. good for ODP" : "background-color: #87cb67",
        "2. improve" : "background-color: #fefebf",
        "1. update" : "background-color: #f78c51"
    }

flag_colours = {
        True : "color:red",
        False : "color:green"
    }

def make_color_mask_dataset_lpa(df):
    #DataFrame with same index and columns names as original filled empty strings
    df_color_map =  pd.DataFrame("", index=df.index, columns=df.columns)
    # turn label column into colours
    df_color_map["quality_level_label"] = df["quality_level_label"].map(level_colours)

    flag_slice = df.columns[3:-1]
    for s in flag_slice:
        df_color_map[s] = df[s].map(flag_colours)

    return df_color_map


def get_dataset_lpa_summary(dataset):
    """
    Given a dataset, will get a dataset subset from qual_cat_summary, 
    pivot it to have each quality category as a column, and then join
    on the overall quality score from qual_summary.
    """

    # subset and pivot
    df_wide = qual_cat_summary[
        qual_cat_summary["pipeline"] == dataset
    ].pivot(
        columns = "quality_category",
        values = "issue_flag",
        index = ["pipeline", "organisation", "organisation_name"]
    ).reset_index()

    # join on the quality category for the provision
    df_wide_flagged = df_wide.merge(
        qual_summary[["pipeline", "organisation", "quality_level_label"]],
        how = "left",
        on = ["pipeline", "organisation"]
    )

    return df_wide_flagged.style.apply(make_color_mask_dataset_lpa, axis=None)


dataset_list = qual_cat_summary["pipeline"].sort_values().drop_duplicates().values

dataset_dropdown = widgets.Dropdown(
    options = dataset_list,
    value = "conservation-area",
    description = "Select Dataset: ",
)

## Chart

In [20]:
# VISUALISE

# qual_summary_subset = qual_summary[qual_summary["pipeline"].isin(subset_bfl)]
# qual_summary_subset = qual_summary[qual_summary["pipeline"].isin(subset_dvl)]

# color map to use in chart
cmap = plt.get_cmap('RdYlGn')
colors = [cmap(i / 4) for i in np.arange(1, 5)]

def make_quality_overview_chart(subset):
    """
    Uses the qual summary table to display a horizontal bar chart 
    """

    qual_summary_subset = qual_summary[qual_summary["pipeline"].isin(dataset_subset_dict[subset])]

    # count providers by dataset & quality level
    qual_chart = qual_summary_subset.groupby(["pipeline", "quality_level", "quality_level_label"], as_index=False).agg(
        n_providers = ("quality_level", "count")
    )

    qual_chart.sort_values(["pipeline", "quality_level_label"], inplace=True)
    qual_chart_wide = qual_chart.pivot(columns = "quality_level_label", values = "n_providers", index = "pipeline")
    
    qual_chart_wide.plot.barh(
        stacked = True, 
        color = colors, 
        figsize = (9, 6))

    # Add labels and title
    plt.xlabel('Count of providers')
    plt.ylabel('Dataset')
    plt.title('Quality levels for ODP datasets')
    plt.legend(title='Quality level')

    return plt.show()


subset_dropdown = widgets.Dropdown(
    options = dataset_subset_dict.keys(),
    # value = dataset_list[0],
    description = "Select Dataset subset: ",
)

# widgets.interact(make_quality_overview_chart, subset = subset_dropdown)

# Reports

### Data quality overview chart - by dataset groups

In [21]:
widgets.interact(make_quality_overview_chart, subset = subset_dropdown)

interactive(children=(Dropdown(description='Select Dataset subset: ', options=('ODP', 'BFL', 'Developers'), va…

<function __main__.make_quality_overview_chart(subset)>

### ODP LPA overview table by dataset & quality

In [22]:
odp_lpa_summary.style.apply(make_color_mask_odp_lpa, axis=None)

pipeline,organisation,organisation_name,article-4-direction,article-4-direction-area,conservation-area,conservation-area-document,listed-building-outline,tree,tree-preservation-order,tree-preservation-zone
0,local-authority:ASF,Ashford Borough Council,no data,no data,no data,no data,4. excellent,no data,4. excellent,no data
1,local-authority:BDG,London Borough of Barking and Dagenham,no data,2. improve,4. excellent,no data,4. excellent,no data,no data,no data
2,local-authority:BIR,Birmingham City Council,4. excellent,4. excellent,4. excellent,4. excellent,4. excellent,no data,no data,2. improve
3,local-authority:BNE,London Borough of Barnet,4. excellent,4. excellent,4. excellent,3. good for ODP,4. excellent,4. excellent,3. good for ODP,4. excellent
4,local-authority:BOL,Bolton Metropolitan Borough Council,no data,no data,1. update,no data,no data,no data,no data,no data
5,local-authority:BST,Bristol City Council,4. excellent,3. good for ODP,4. excellent,4. excellent,3. good for ODP,4. excellent,4. excellent,4. excellent
6,local-authority:CAS,Castle Point Borough Council,4. excellent,4. excellent,4. excellent,4. excellent,4. excellent,no data,no data,no data
7,local-authority:CAT,Canterbury City Council,no data,3. good for ODP,4. excellent,no data,3. good for ODP,no data,no data,no data
8,local-authority:DOV,Dover District Council,4. excellent,3. good for ODP,4. excellent,no data,4. excellent,4. excellent,3. good for ODP,2. improve
9,local-authority:ECA,East Cambridgeshire District Council,4. excellent,4. excellent,4. excellent,no data,no data,no data,no data,no data


### Dataset quality scoring detail table

In [23]:
widgets.interact(get_dataset_lpa_summary, dataset = dataset_dropdown)

interactive(children=(Dropdown(description='Select Dataset: ', index=15, options=('agricultural-land-classific…

<function __main__.get_dataset_lpa_summary(dataset)>

#### Quality investigations

In [ ]:
qual_summary_subset[qual_summary_subset["pipeline"] == "tree"]

In [ ]:
qual_chart.groupby(["quality_level", "quality_level_label"], as_index = False).agg(
    total = ("n_providers", "sum")
)

In [ ]:
# Check some specifics - conservation area "update"

dataset = "conservation-area"
level = 4

level_provisions = ep_res_qual_all[
    (ep_res_qual_all["pipeline"] == dataset) & 
    (ep_res_qual_all["quality_level"] == level)][
        ["pipeline", "organisation"]
    ].drop_duplicates()

level_prov_all_issues = level_provisions.merge(
    ep_res_qual_all,
    how = "left", 
    on = ["pipeline", "organisation"]
    )

level_prov_all_issues[level_prov_all_issues["quality_level"].notnull()].drop_duplicates().groupby(["issue_type"], as_index=False).size()

In [ ]:
# Check some specifics - conservation area "update"

dataset = "tree-preservation-zone"
level = 3

level_provisions = qual_summary[
    (qual_summary["pipeline"] == dataset) & 
    (qual_summary["quality_level"] == level)][
        ["pipeline", "organisation"]
    ].drop_duplicates()

level_prov_all_issues = level_provisions.merge(
    ep_res_qual_all,
    how = "left", 
    on = ["pipeline", "organisation"]
    )

level_prov_all_issues[level_prov_all_issues["quality_level"].notnull()].drop_duplicates().groupby(["issue_type"], as_index=False).size()

### Provision view

In [ ]:
prov_summary_now = ep_res_issues[["organisation", "organisation_name", "collection", "pipeline"]].drop_duplicates().groupby(
    ["collection", "pipeline"], as_index=False
    ).agg(
    n_providers = ("organisation", "size")
)

prov_summary_now.head(10)

In [ ]:
# get table of active endpoints and resources, with issue summaries per resource joined on
q = f"""
    SELECT 
        organisation, organisation_name, dataset, active_endpoint_count,
        CASE WHEN active_endpoint_count > 0 then 1 else 0 end as active_endpoint_flag
    FROM provision_summary 
"""

prov_all = query_sqlite(os.path.join(data_dir, "performance.db"), q)
prov_all

#### Current

In [ ]:
prov_summary_now = prov_all.groupby(
    ["dataset"], as_index=False
    ).agg(
    n_providers = ("active_endpoint_flag", "sum"),
    n_expected = ("active_endpoint_flag", "size")
)

prov_summary_now["trusted_data"] = prov_summary_now["n_providers"]
prov_summary_now["no_data"] = prov_summary_now["n_expected"] - prov_summary_now["n_providers"]

prov_summary_now.head()

In [ ]:
prov_summary_chart = prov_summary_now[prov_summary_now["dataset"].isin(subset_odp)]

prov_summary_chart[["dataset", "trusted_data", "no_data"]].plot.bar(x = "dataset", stacked = True, color = ["black", "#d4d4d4"])

plt.ylabel('n providers')
plt.title(f'Current provision for ODP datasets')
plt.legend(title='Data type')
plt.show()

#### Historic

In [ ]:
def get_active_endpoints(dataset, date):

    q = f"""
        select distinct pipeline as dataset, name, DATE('{date}') as active_date, endpoint_entry_date, endpoint_end_date
        from reporting_historic_endpoints 
        where
            pipeline = '{dataset}'
            and endpoint_entry_date <= DATE('{date}')
            and (
                endpoint_end_date > DATE('{date}')
                or endpoint_end_date = "")
"""
    
    df = query_sqlite(os.path.join(data_dir, "performance.db"), q)

    return df

# test = get_active_endpoints("tree-preservation-zone", "2024-07-11")
# test

In [ ]:
dataset = "brownfield-land"

date_range = pd.date_range(end = "20241001", periods = 24, freq = "MS").format("%Y-%m-%d")

prov_hist = [get_active_endpoints(dataset, d) for d in date_range[1:-1]]
prov_hist_df = pd.concat(prov_hist)

prov_summary_hist = prov_hist_df[["dataset", "name", "active_date"]].drop_duplicates().groupby(["active_date", "dataset"], as_index = False).agg(
    n_providers = ("dataset", "size")
)

print(len(prov_summary_hist))
prov_summary_hist = prov_summary_hist.merge(
    prov_summary_now[["dataset", "n_expected"]],
    how = "left",
    on = "dataset"
)

prov_summary_hist["trusted_data"] = prov_summary_hist["n_providers"]
prov_summary_hist["no_data"] = prov_summary_hist["n_expected"] - prov_summary_hist["n_providers"]

print(len(prov_summary_hist))
# prov_summary_hist
prov_summary_hist[["dataset", "active_date", "trusted_data", "no_data"]].plot.area(x = "active_date", stacked = True, color = ["black", "#d4d4d4"])

plt.xlabel('date')
plt.ylabel('n providers')
plt.title(f'Historic provision for: {dataset}')
plt.legend(title='Data type')
plt.show()